TypeError: koordinaatit() takes no arguments

In [88]:
import math
def calculate_perimeter(a,b):
    perimeter = math.pi * ( 3*(a+b) - math.sqrt( (3*a + b) * (a + 3*b) ) )
    return perimeter

x=stellar_system.xcoord[1]
y=stellar_system.xcoord[1]
calculate_perimeter(x,y)

942477796076.938

In [1]:
import numpy as np
import copy
'''
Class file of celestial objects. Also declaring of objects are included

    objectlist and cometlist attributes:
        Name (str):       Name of the object
        Mass (float):     Mass of the object in kilograms
        Distance (list):  x and y coordinates of the object in meters. 
                          It is calculated as distance to the zeroth object (not the center of mass).
        Velocity (list):  x and y velocities in meters/second
        Track (list):     Record of trajectory of the object in shape (:,2)
        color (str):      Color of the object in animation
        size (int):       Size of the object in animation
        radius (float):   Radius of the object in meters.
        
    Variables:
        Au (float):       Distance from Sun to Earth in meters. 1 Au = 1 Astronomical unit
'''

Au = 150e9 #Distance to the Sun from Earth.
numcom = 5 # Number of comets to be generated
class objectlist:
    name          = ""
    mass          = 0 #kg
    distance      = [0,0] #In Au, works as coordinates
    velocity      = [0,0] #In Au?
    track         = [0,0]
    color         = ""
    size          = 1
    radius        = 10 #meters
    pass
class comets:
    name = "Comet"
    mass = np.random.uniform(1e13,1e17)
    distance = [np.random.uniform(20,35),0]
    velocity = [0,np.random.uniform(1000,25000)]
    track = [0,0]
    color = "grey"
    size = 1
    radius = 10
    
Mars = objectlist()
Mars.name="Mars"
Mars.mass=6.39e23 
Mars.distance=[1.666*Au, 0]
Mars.velocity=[0,21970]
Mars.track = copy.deepcopy(Mars.distance)
Mars.color = "darkred"
Mars.size = 5
Mars.radius = 3400e3

Jupiter = objectlist()
Jupiter.name="Jupiter"
Jupiter.mass= 1.898e27
Jupiter.distance=[5.46*Au, 0]
Jupiter.velocity=[0,12400]
Jupiter.track = copy.deepcopy(Jupiter.distance)
Jupiter.color = "orange"
Jupiter.size = 20
Jupiter.radius = 70000e3

Earth = objectlist()
Earth.name="Earth"
Earth.mass= 5.972e24
Earth.distance=[1.0167*Au, 0]
Earth.velocity=[0,29290]
Earth.track = copy.deepcopy(Earth.distance)
Earth.color = "blue"
Earth.size = 5
Earth.radius = 6400e3

i=0 #Make comets
comet = np.array([comets() for _ in range(numcom)])
while i<len(comet):
    comet[i].name = "Comet"+str(i)
    comet[i].mass = np.random.uniform(1e14,1e16)
    comet[i].distance = [np.random.uniform(30,45)*Au,0]
    comet[i].velocity = [np.random.uniform(-10000,-25000),-1000]
    comet[i].track = copy.deepcopy(comet[i].distance)
    comet[i].size = int(np.random.uniform(1,3))
    comet[i].radius = np.random.uniform(5e3,50e3)
    i=i+1

Saturn = objectlist()
Saturn.name = "Saturn"
Saturn.mass = 5.683e26
Saturn.distance = [10.1238*Au,0]
Saturn.velocity = [0, 9090]
Saturn.track = copy.deepcopy(Saturn.distance)
Saturn.color = "orange"
Saturn.size = 16
Saturn.radius = 60000e3

Sun = objectlist()
Sun.name = "Sun"
Sun.mass = 2.0e30 
Sun.distance = [0,0]
Sun.velocity = [0,0]
Sun.track = copy.deepcopy(Sun.distance)
Sun.color = "yellow"
Sun.size = 40
Sun.radius = 700000e3

BH = objectlist()
BH.name ="Black hole"
BH.mass = 8*Sun.mass
BH.distance = [0, 10*Au]
BH.velocity = [-10000,0]
BH.track = copy.deepcopy(BH.distance)
BH.color = "black"
BH.size = 5
BH.radius = 50e3
#Comet, Saturn, SMBH, BH

BD = objectlist()
BD.name="Brown Dwarf"
BD.mass =5*Jupiter.mass
BD.distance =[10*Au,0]
BD.velocity = [-1000,1000]
BD.track = copy.deepcopy(BD.distance)
BD.color = "brown"
BD.size = 30
BD.radius = 80000e3


# Center of mass coordinates
# Parameters defined from main program
CM = objectlist()
CM.name = "Center of Mass"
CM.mass = 0
CM.distance = [0,0]
CM.velocity = [0,0]
CM.track = copy.deepcopy(CM.distance)
CM.color = "darkgrey"
CM.size = 5


In [ ]:
import numpy as np
import copy
import scipy
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
ctime,ci = [],[]
l_obj = [Sun,Earth,Mars,BD,Jupiter,Saturn,comet[0],comet[1],comet[2]] #,Earth,Jupiter] #,comet[0]]
#get maxdistance (assuming the last object is farthest):
maxdist = np.max(l_obj[-1].distance)
print(maxdist)
i=0
#return track length to zero from possible previous simulations, also set previous starting point.
while i<len(l_obj):
    l_obj[i].track = copy.deepcopy(l_obj[i].distance)
    i=i+1

#PRINT
print("In the beginning track: ", l_obj[0].track)

lengt = len(l_obj)
#List of constants in program
year = 24*60*60*365
simulength = int(1000*year)
Gra    = 6.67e-11
M=0
dt=24*60*60*1 #one day cycle
for i in range(len(l_obj)):
    M = M + l_obj[i].mass
CM.mass = M
l_con = [Gra,Au,year,M]



def dist(l_obj,i,j,k):
    return (l_obj[i].distance[k] - l_obj[j].distance[k])
    
def distance_sq(l_obj,i,j):
    x = dist(l_obj,i,j,0) #l_obj[i].distance[0] - l_obj[j].distance[0]
    y = dist(l_obj,i,j,1) #l_obj[i].distance[1] - l_obj[j].distance[1]
    return (x**2 + y**2)**1.5 #Maybe change this 0.5?
def gravity(l_obj,l_con):
    k=0
    p=0
    G_totx=[]
    G_toty=[]
    while k<len(l_obj):
        l_Gx=[]
        l_Gy=[]
        while p<len(l_obj):
            if p==k:
                p=p+1
                if p>=len(l_obj):
                    break
            l_Gx = np.append(l_Gx, -Gra*dist(l_obj,k,p,0)*(l_obj[k].mass*l_obj[p].mass)/(distance_sq(l_obj,k,p)))
            l_Gy = np.append(l_Gy, -Gra*dist(l_obj,k,p,1)*(l_obj[k].mass*l_obj[p].mass)/(distance_sq(l_obj,k,p)))
            p=p+1
        G_totx = np.append(G_totx, np.sum(l_Gx))
        G_toty = np.append(G_toty, np.sum(l_Gy))
        k=k+1
        p=0
    return G_totx,G_toty

def com(l_obj,l_con): #Center of mass distance to the sun
    i=0
    tempx=0
    tempy=0
    while i<len(l_obj): #Vapausasteista riippumaton lasku
        tempx = tempx + (l_obj[i].mass*l_obj[i].distance[0])
        tempy = tempy + (l_obj[i].mass*l_obj[i].distance[1])
        i=i+1
    x0 = tempx / CM.mass
    y0 = tempy / CM.mass
    return x0,y0
#----------------------
def move(l_obj,l_con,dt): #Move the objects dt
    #Count new velocity, move the object, update position
    GS = gravity(l_obj,l_con)
    px = []
    py = []
    i=0
    while i<len(l_obj):
        velox = l_obj[i].velocity[0] + (GS[0][i]*dt/l_obj[i].mass)
        veloy = l_obj[i].velocity[1] + (GS[1][i]*dt/l_obj[i].mass)
        px = np.append(px,velox*dt) #Travelled distance
        py = np.append(py,veloy*dt)
        l_obj[i].velocity[0],l_obj[i].velocity[1] = velox,veloy       
        
        i=i+1
    #print(px,py)
    return px,py
def simulate(a):

    return 

def update(l_obj,pos,time):
    global ctime,ci
    i=0
    while i<len(l_obj):
        l_obj[i].distance[0] = l_obj[i].distance[0] + pos[0][i]
        l_obj[i].distance[1] = l_obj[i].distance[1] + pos[1][i]
        CM.distance = com(l_obj,l_con)
        #arvo = copy.deepcopy(l_obj[i].distance[0])
        #print(arvo)
        l_obj[i].track = np.vstack((l_obj[i].track,[l_obj[i].distance[0],l_obj[i].distance[1]]))
        CM.track = np.vstack((CM.track,[CM.distance[0],CM.distance[1]]))
        check,index = collision_check(l_obj)
        if check:
            #Indeksi tähän, ja objektin nimitunniste
            print("Jupiiiii", time)
            ctime = np.append(ctime,time)
            ci = np.append(ci,index) #index of lighter object.
        i=i+1
    return 

def collision_check(l_obj):
    i=0
    j=0
    exit = False
    #print('{:.2e}'.format(abs(l_obj[0].distance[1])),'{:.2e}'.format( abs(l_obj[5].distance[1])))
    while i<len(l_obj):
        while j<len(l_obj):
            if i==j:
                j=j+1
                if j>=len(l_obj):
                    break
            if (abs(l_obj[i].distance[0]-l_obj[j].distance[0])<10*(l_obj[i].radius+l_obj[j].radius)
               and abs(l_obj[i].distance[1]-l_obj[j].distance[1])<10*(l_obj[i].radius+l_obj[j].radius)):
                print("TÖRMÄYS", i,j)
                #Get the larger value!
                new_mass = l_obj[i].mass + l_obj[j].mass
                print("First velocity ", BH.velocity)
                new_velox = (l_obj[i].velocity[0]*l_obj[i].mass + l_obj[j].velocity[0]*l_obj[j].mass)/new_mass
                new_veloy = (l_obj[i].velocity[1]*l_obj[i].mass + l_obj[j].velocity[1]*l_obj[j].mass)/new_mass
                t = max(l_obj[i].mass,l_obj[j].mass)
                s = min(l_obj[i].mass,l_obj[j].mass)
                kr=0
                print("Then: ", BH.velocity)
                while kr<len(l_obj):
                    if l_obj[kr].mass == t:
                        l_obj[kr].mass = new_mass
                        l_obj[kr].velocity = [new_velox,new_veloy]
                        tr=0
                        while tr<len(l_obj):
                            if l_obj[tr].mass==s:
                                l_obj[tr].mass = 1
                                #l_obj[tr] = copy.deepcopy(zero)
                                #print(l_obj[tr].distance,l_obj[tr].mass)
                                #l_obj = np.delete(l_obj,tr)
                                #print("TÄÄLLÄ",tr, "massbh: ", l_obj[-1].mass, "lenght: ", len(l_obj))
                                #record the collision time and objects, transfer to animation.
                                #From now on, object will have zero mass and no movement.
                                #TEE BOOLELAINEN ATTRIBUUTTI. Jos se on päällä,
                                #kerää pelkkä aika ja indeksi.
                                exit=True
                                return exit,tr
                            tr=tr+1
                    if exit:
                        break
                    kr=kr+1
                    
                
            j=j+1
                #delete object.
        i=i+1
        j=0
    return False,False

#SIMUlOI:

i=0
while i<simulength:
    time = int(i/dt)
    simulate(update(l_obj,move(l_obj,l_con,dt),time))
    
    i=i+dt
anilength=int(i/dt)
i=0
while i<len(ctime)-1:
    if ctime[i]==ctime[i+1]:
        ctime = np.delete(ctime,i)
        ci = np.delete(ci,i)
        i=i-1

    i=i+1
print("täällä")
try:
    ctime=ctime.astype(int)
    ci = ci.astype(int)
except:
    ctime=[0]
    ci = [0]
    print("try:", ctime,ci)
print(ci, type(ci), ci[0])
print(CM.track)

print("data ready")
print("datapoints: ", len(l_obj[0].track[:,0]))


i=0

#reshape data for animation!!:
i=1
j=0
tes=0
while j<len(l_obj):
    l_obj[j].track = l_obj[j].track[9::10,:]    
    j=j+1
anilength = len(l_obj[0].track)


#Mistä simulaatiota ohjataan?
#Animaatioalustukset
i=0
#Set simulation list parameters.
fig, ax = plt.subplots(figsize=(10,10))
ax.set_aspect('equal')
ax.grid()
default_x_ticks = np.linspace(-10*Au,10*Au,6)
plt.xticks(default_x_ticks, [-10,-6,-2,2,6,10])
default_y_ticks = np.linspace(-10*Au,10*Au,6)
plt.yticks(default_y_ticks, [-10,-6,-2,2,6,10])
ax.set_xlim(-12*Au,12*Au)
ax.set_ylim(-12*Au,12*Au)
ax.set_xlabel("Distance [AU]")
ax.set_ylabel("Distance [AU]")
#-----------------------------
my_vars = {}
for i in range(len(l_obj)+1):
    line = "line%d" % i
    point = "point%d" % i
    text = "text%d" % i
    my_vars["xdata"+str(i)] = []
    my_vars["ydata"+str(i)] = []
    my_vars[line] = i
    my_vars[point] = i
    my_vars[text] = i
    try:
        my_vars["line"+str(i)], = ax.plot([],[],'-g',lw=1, linestyle = "dashed")
        my_vars["point"+str(i)], = ax.plot(l_obj[i].distance[0],l_obj[i].distance[1], 
                                       marker="o", markersize=0.5*l_obj[i].size, 
                                       markeredgecolor=l_obj[i].color, markerfacecolor=l_obj[i].color)
        my_vars["text"+str(i)] = ax.text(l_obj[i].distance[0],l_obj[i].distance[1] , l_obj[i].name)
    except:
        my_vars["line"+str(i)], = ax.plot([CM.distance[0]],[CM.distance[1]],'-g',lw=1, linestyle = "dashed")
        my_vars["point"+str(i)], = ax.plot(CM.distance[0],CM.distance[1], 
                                       marker="o", markersize=CM.size, 
                                       markeredgecolor=CM.color, markerfacecolor=CM.color)
        my_vars["text"+str(i)] = ax.text(CM.distance[0],CM.distance[1] , CM.name)
        
        #zero my_vars?:

    
def a_update(i):
    global ctime,ci
    
    j=0
    my_vars["xdata"+str(len(l_obj))].append(CM.track[i,0])
    my_vars["ydata"+str(len(l_obj))].append(CM.track[i,1])
    my_vars["point"+str(len(l_obj))].set_data(CM.track[i,0],CM.track[i,1])
    #my_vars["line"+str(len(l_obj))].set_data(my_vars["xdata"+str(len(l_obj))],my_vars["ydata"+str(len(l_obj))])
    my_vars["text"+str(len(l_obj))].set_position((CM.track[i,0],CM.track[i,1]))

    k=0
    while k<len(ctime):
        if i-ctime[k]==-1:
            #Explosion!
            #my_vars["line"+str(ci[k])], = ax.plot([],[],'-g',lw=1, linestyle = "None")
            #my_vars["point"+str(ci[k])], = ax.plot(l_obj[ci[k]].distance[0],l_obj[ci[k]].distance[1], 
              #                        marker="o", markersize=1*l_obj[ci[k]].size, 
             #                         markeredgecolor="black", markerfacecolor="black")
            #my_vars["text"+str(ci[k])] = ax.text(l_obj[ci[k]].distance[0],l_obj[ci[k]].distance[1]
               #                                    , "tetsts", color ="None")
            my_vars["line"+str(ci[k])].set_data(my_vars["xdata"+str(ci[k])],my_vars["ydata"+str(ci[k])])
            my_vars["point"+str(ci[k])].set_data(-1000*Au,1000*Au)
            my_vars["text"+str(ci[k])].set_position((-1000*Au,1000*Au))            
            l_obj[ci[k]].track = [s * 0 for s in l_obj[ci[k]].track]
            
            
            
            #change the objects colour transparent
            #l_obj[ci[k]].color = "blue"
            #l_obj[ci[k]].size = 1
            #l_obj[ci[k]].name = ""
            
            
            #del my_vars["line"+str(ci[k])]
            #del my_vars["point"+str(ci[k])]
            #del my_vars["text"+str(ci[k])]
            print("TESTI VAIN")    
        k=k+1
        #Tänne ehto: Jos attribuutti on true, linjaväri, objektin väri muuttuu valkoiseksi, 
        #if "getcollison"
    while j<len(l_obj):
        try:
            my_vars["xdata"+str(j)].append(l_obj[j].track[i,0])
            my_vars["ydata"+str(j)].append(l_obj[j].track[i,1])        
        
            my_vars["line"+str(j)].set_data(my_vars["xdata"+str(j)],my_vars["ydata"+str(j)])
            my_vars["point"+str(j)].set_data(l_obj[j].track[i,0],l_obj[j].track[i,1])
            my_vars["text"+str(j)].set_position((l_obj[j].track[i,0],l_obj[j].track[i,1]))
            
        except:
            j=j+1
            continue
        j=j+1
    if i%100 == 0:
        print("Animating frame ", i)
    #print("FRAME NUMBER", i)

    ax.axis('equal')
    ax.set_xlim(-12*Au,12*Au)
    ax.set_ylim(-12*Au,12*Au)

    return my_vars["line1"],my_vars["point1"],my_vars["text1"]

anim = animation.FuncAnimation(fig,func=a_update,frames=int(anilength),interval=10,blit=True)

from IPython.display import HTML

#writervideo = animation.FFMpegWriter(fps=60)
#anim.save('increasingStraightLine.mp4', writer=writervideo)

HTML(anim.to_jshtml())

5327556940965.86
In the beginning track:  [0, 0]
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3046
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3046
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3047
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3048
TÖRMÄYS 0 7
First velocity  [-10000, 0]
Then:  [-10000, 0]
Jupiiiii 3048
TÖ

/tmp/ipykernel_7527/4229733311.py:242: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-g" (-> linestyle='-'). The keyword argument will take precedence.
  my_vars["line"+str(i)], = ax.plot([],[],'-g',lw=1, linestyle = "dashed")
/tmp/ipykernel_7527/4229733311.py:248: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-g" (-> linestyle='-'). The keyword argument will take precedence.
  my_vars["line"+str(i)], = ax.plot([CM.distance[0]],[CM.distance[1]],'-g',lw=1, linestyle = "dashed")
/tmp/ipykernel_7527/4229733311.py:263: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  my_vars["point"+str(len(l_obj))].set_data(CM.track[i,0],CM.track[i,1])
/tmp/ipykernel_7527/4229733311.py:303: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two mino

Animating frame  0
Animating frame  0
Animating frame  0
Animating frame  0
Animating frame  100
Animating frame  200
Animating frame  300
Animating frame  400
Animating frame  500
Animating frame  600
Animating frame  700
Animating frame  800
Animating frame  900
Animating frame  1000
Animating frame  1100
Animating frame  1200
Animating frame  1300
Animating frame  1400
Animating frame  1500
Animating frame  1600
Animating frame  1700
Animating frame  1800
Animating frame  1900
Animating frame  2000
Animating frame  2100
Animating frame  2200
Animating frame  2300
Animating frame  2400
Animating frame  2500
Animating frame  2600
Animating frame  2700
Animating frame  2800
Animating frame  2900
Animating frame  3000


/tmp/ipykernel_7527/4229733311.py:278: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  my_vars["point"+str(ci[k])].set_data(-1000*Au,1000*Au)


TESTI VAIN
TESTI VAIN
TESTI VAIN
Animating frame  3100
Animating frame  3200
Animating frame  3300
Animating frame  3400
Animating frame  3500
Animating frame  3600
Animating frame  3700
Animating frame  3800
Animating frame  3900
Animating frame  4000
Animating frame  4100
Animating frame  4200
Animating frame  4300
Animating frame  4400
Animating frame  4500
Animating frame  4600
Animating frame  4700
Animating frame  4800
Animating frame  4900
Animating frame  5000
Animating frame  5100
Animating frame  5200
Animating frame  5300
Animating frame  5400
Animating frame  5500
Animating frame  5600
Animating frame  5700
Animating frame  5800
Animating frame  5900
Animating frame  6000
Animating frame  6100
Animating frame  6200
Animating frame  6300
Animating frame  6400
Animating frame  6500
Animating frame  6600
Animating frame  6700
Animating frame  6800
Animating frame  6900
Animating frame  7000
Animating frame  7100
Animating frame  7200
Animating frame  7300
Animating frame  7400

In [ ]:
#plotteja
'''
i=0
testx=[]
testy=[]
testx2=[]
testy2=[]
testx3=[]
testy3=[]
testx4=[]
testy4=[]
while i<simulength:
    simulate(update(l_obj,move(l_obj,l_con,dt)))
    
    #plot_testers
    testx = np.append(testx,l_obj[1].distance[0])
    testy = np.append(testy,l_obj[1].distance[1])
    testx2 = np.append(testx2,l_obj[0].distance[0])
    testy2 = np.append(testy2,l_obj[0].distance[1])
    testx3 = np.append(testx3,l_obj[3].distance[0])
    testy3 = np.append(testy3,l_obj[3].distance[1])
    testx4 = np.append(testx4,l_obj[2].distance[0])
    testy4 = np.append(testy4,l_obj[2].distance[1])    
    
    i=i+dt
#Testplots
plt.title(l_obj[1].name)
plt.plot(testx,testy)
plt.plot(testx2,testy2,color="Yellow")
plt.scatter(Sun.distance[0],Sun.distance[1],color="Black")
plt.plot(testx3,testy3,color="black")
plt.plot(testx4,testy4,color="Red")
plt.xlim(-6*Au,6*Au)
plt.show()
'''

In [507]:

#how many variables has to be set up:
my_vars = {}
for i in range(len(l_obj)):
    var_name = "var%d" % i
    my_vars[var_name] = i

print(my_vars["var3"])
print(type(my_vars["var1"]))
my_vars["var1"] = 100
print(my_vars["var0"])


3
<class 'int'>
0


In [510]:
my_vars = {}
for i in range(len(l_obj)):
    line = "line%d" % i
    point = "point%d" % i
    text = "text%d" % i
    my_vars[line] = i
    my_vars[point] = i
    my_vars[text] = i
print(my_vars["text1"])
my_vars["text1"]="jee"
print(my_vars["text1"])

1
jee


In [50]:
for i in range(len(l_obj)):
    print(l_obj[i].name)
l_obj = np.delete(l_obj,3)
for i in range(len(l_obj)):
    print(l_obj[i].name)


Sun
Earth
Mars
Black hole
Sun
Earth
Mars


In [295]:
#
testarray = [3,2,5,7,9]
testarray = [s * 1e9 for s in testarray]
print(testarray)

[3000000000.0, 2000000000.0, 5000000000.0, 7000000000.0, 9000000000.0]


In [91]:
i=1
j=0
tes=0
while j<len(l_obj):
    l_obj[j].track = l_obj[j].track[9::10,:]    
    j=j+1
print(np.shape(l_obj[0].track))

(182, 2)
